<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/Trial_26_06_2024_Timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Code Progression:**

Enter line ID, direction.

It gives the origination and destination of the lineID and writes the  stop points between origination and destination in the sequence as they appears in the given direction of travel.

Asks the user to enter the no of QSI points and the row number corresponding to them from the csv file.

Returns the arrival predictions and summary metrics of the given QSI points.

**Program User Input Sequence**

**1. Enter line ID:**

**2. Enter direction (e.g., inbound or outbound):**

After this the origination and destination details of the given lineID and direction is displayed on screen.

Then the message "Data written to 'stop_details.csv' successfully." appears with "Data from CSV:" listed on the screen.

**3. Enter the number of QSI stop points:**

**4. Enter the row number for QSI stop point 1:**
   Enter the row number for QSI stop point 2:
   Enter the row number for QSI stop point 3:
   Enter the row number for QSI stop point 4:

**Test input: For 1 stop point:**
line ID:141
direction: outbound
After the stop points are displayed on screen, find (Ctrl + F) 490015195M. Enter the row number corresponding to this stop point.

**Test input: For 2 stop point:**
line ID:214
direction: inbound
After the stop points are displayed on screen, find (Ctrl + F) 490010012N, and 490000036L. Enter the row numbers corresponding to these stop points.

**Test input: For 3 stop point:**
line ID:22
direction: outbound
After the stop points are displayed on screen, find (Ctrl + F) 490012375E, 490009370E and 490008435E. Enter the row numbers corresponding to these stop points.

**Test input: For 4 stop point:**
line ID:444
direction: outbound
After the stop points are displayed on screen, find (Ctrl + F) 490007773E, 490012619L, 490015834E and 490006564N. Enter the row numbers corresponding to these stop points.

**Test input: For 5 stop point:**
line ID:179
direction: outbound
After the stop points are displayed on screen, find (Ctrl + F) 490015202N, 490012012E, 490005183E, 490007380S and 490007021E. Enter the row numbers corresponding to these stop points.


In [ ]:
# Step 1: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 2: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.4 MB/s eta 0:00:00


In [ ]:
# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - Fetch Error",
                    'Arrival Predictions': "No data available",
                    'Summary Metrics': "No data available"
                })

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Moorfields Eye Hospital
Arrival Predictions:
Line Vehicle ID Stop Point Direction Expected Arrival (BST)  Headway (minutes)  AWT/bus (minutes)  WAWT
 214    LA19KBY 490010012N   inbound               09:13:00                0.0                0.0   0.0
 214    LA19KBP 490010012N   inbound               09:20:00                7.0                3.5  24.5
 214    LA19KAO 490010012N   inbound               09:27:00                7.0                3.5  24.5
 214    LA19KBX 490010012N   inbound               09:34:00                7.0                3.5  24.5

Summary Metrics:
                                                       Metric  Value
                    Number of buses scheduled per hour (nbph)    6.0
                                         Total WAWT (minutes)   73.5
Time difference between 1st and last observed buses (minutes)   21.0
                                                AWT (minutes)    3.5
                                                SWT 

Including the number of buses observed at each QSI point

In [ ]:
import csv
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                num_buses_observed = df['Vehicle ID'].nunique()
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)', 'Number of buses observed'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt, num_buses_observed]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - Fetch Error",
                    'Arrival Predictions': "No data available",
                    'Summary Metrics': "No data available"
                })

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Moorfields Eye Hospital
Arrival Predictions:
Line Vehicle ID Stop Point Direction Expected Arrival (BST)  Headway (minutes)  AWT/bus (minutes)  WAWT
 214    LA19KAK 490010012N   inbound               09:59:00                0.0                0.0   0.0
 214    LA19JZW 490010012N   inbound               10:06:00                7.0                3.5  24.5
 214    LA19KBE 490010012N   inbound               10:13:00                7.0                3.5  24.5
 214    LA19KBZ 490010012N   inbound               10:22:00                9.0                4.5  40.5

Summary Metrics:
                                                       Metric  Value
                    Number of buses scheduled per hour (nbph)   6.00
                                         Total WAWT (minutes)  89.50
Time difference between 1st and last observed buses (minutes)  23.00
                                                AWT (minutes)   3.89
                                                SWT 

KeyboardInterrupt: 

Displaying the number of buses observed after nbph under the summary metrics --> File share 4

In [ ]:
import csv
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                num_buses_observed = df['Vehicle ID'].nunique()
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Number of buses observed', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, num_buses_observed, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - Fetch Error",
                    'Arrival Predictions': "No data available",
                    'Summary Metrics': "No data available"
                })

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Fetch Error
Arrival Predictions:
No data available

Summary Metrics:
No data available

--------------------------------------------

QSI Stop Point: 2 - Fetch Error
Arrival Predictions:
No data available

Summary Metrics:
No data available

--------------------------------------------

QSI Stop Point: 3 - Fetch Error
Arrival Predictions:
No data available

Summary Metrics:
No data available

--------------------------------------------



KeyboardInterrupt: 

Fetching Timetable_26_06_2024

In [ ]:
import requests
import pandas as pd

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': entry['hour'],
                    'Minute': entry['minute'],
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)
        df = df[['Time', 'Line ID', 'Stop Point ID', 'Direction']]

        print("\nTimetable:")
        print(df.to_string(index=False))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 214
Enter stop point ID: 490010012N
Enter direction (e.g., inbound or outbound): inbound


ValueError: Unknown format code 'd' for object of type 'str'

TIME TABLE OF BUSES

In [ ]:
import requests
import pandas as pd

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': int(entry['hour']),
                    'Minute': int(entry['minute']),
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)
        df = df[['Time', 'Line ID', 'Stop Point ID', 'Direction']]

        print("\nTimetable:")
        print(df.to_string(index=False))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Timetable:
 Time Line ID Stop Point ID Direction
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound
05:17     179    490005183E  Outbound
05:17     179    490005183E  Outbound
05:17     179    490005183E  Outbound
05:37     179    490005183E  Outbound
05:37     179    490005183E  Outbound
05:37     179    490005183E  Outbound
05:57     179    490005183E  Outbound
05:57     179    490005183E  Outbound
05:57     179    490005183E  Outbound
06:17     179    490005183E  Outbound
06:17     179    490005183E  Outbound
06:17     179    490005183E  Outbound
06:34     179    490005183E  Outbound
06:34     179    490005183E  Outbound
06:37     179    490005183E  Outbound
06:42     179    490005183E  Outbound
06:46     179    490005183E  Outbound
06:46     179    490005183E  Outbound
06:57     179    490005183E  Outbound
06:58    

Fetching Time table and dividing to slots-but there are hr s displayed as 24!

In [ ]:
import requests
import pandas as pd

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': int(entry['hour']),
                    'Minute': int(entry['minute']),
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Display timetable by slots
        current_slot = 1
        print("\nTimetable:")
        while current_slot <= 24:
            print(f"\nSlot {current_slot:02d} (Hour {current_slot-1:02d}:00 to Hour {current_slot % 24:02d}:00)")
            slot_df = df[df['Slot'] == current_slot]
            if not slot_df.empty:
                print(slot_df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))
            else:
                print("No entries")
            current_slot += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Timetable:

Slot 01 (Hour 00:00 to Hour 01:00)
 Time Line ID Stop Point ID Direction
24:02     179    490005183E  Outbound
24:02     179    490005183E  Outbound
24:07     179    490005183E  Outbound
24:07     179    490005183E  Outbound
24:22     179    490005183E  Outbound
24:22     179    490005183E  Outbound
24:27     179    490005183E  Outbound
24:27     179    490005183E  Outbound
24:42     179    490005183E  Outbound
24:42     179    490005183E  Outbound
24:47     179    490005183E  Outbound
24:47     179    490005183E  Outbound

Slot 02 (Hour 01:00 to Hour 02:00)
No entries

Slot 03 (Hour 02:00 to Hour 03:00)
No entries

Slot 04 (Hour 03:00 to Hour 04:00)
No entries

Slot 05 (Hour 04:00 to Hour 05:00)
 Time Line ID Stop Point ID Direction
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound

Slot 06 (Hour 05:00 to 

Fetching Time table and dividing to slots-changing 24 to 00

In [ ]:
import requests
import pandas as pd

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': int(entry['hour']),
                    'Minute': int(entry['minute']),
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Display timetable by slots
        current_slot = 1
        print("\nTimetable:")
        while current_slot <= 24:
            slot_start = (current_slot - 1) % 24
            slot_end = current_slot % 24
            print(f"\nSlot {current_slot:02d} (Hour {slot_start:02d}:00 to Hour {slot_end:02d}:00)")
            slot_df = df[df['Slot'] == current_slot]
            if not slot_df.empty:
                print(slot_df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))
            else:
                print("No entries")
            current_slot += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Timetable:

Slot 01 (Hour 00:00 to Hour 01:00)
 Time Line ID Stop Point ID Direction
00:02     179    490005183E  Outbound
00:02     179    490005183E  Outbound
00:07     179    490005183E  Outbound
00:07     179    490005183E  Outbound
00:22     179    490005183E  Outbound
00:22     179    490005183E  Outbound
00:27     179    490005183E  Outbound
00:27     179    490005183E  Outbound
00:42     179    490005183E  Outbound
00:42     179    490005183E  Outbound
00:47     179    490005183E  Outbound
00:47     179    490005183E  Outbound

Slot 02 (Hour 01:00 to Hour 02:00)
No entries

Slot 03 (Hour 02:00 to Hour 03:00)
No entries

Slot 04 (Hour 03:00 to Hour 04:00)
No entries

Slot 05 (Hour 04:00 to Hour 05:00)
 Time Line ID Stop Point ID Direction
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound
04:57     179    490005183E  Outbound

Slot 06 (Hour 05:00 to 

Fetching the Time table of the given stop point of given lineID and direction, and returning the slot timetable with respect to the current time in BST

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': int(entry['hour']),
                    'Minute': int(entry['minute']),
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Get current hour in BST
        bst_tz = pytz.timezone('Europe/London')
        current_time = datetime.now(bst_tz)
        current_hour = current_time.hour

        # Determine current slot
        current_slot = current_hour + 1
        if current_slot > 24:
            current_slot = 1

        # Display timetable for the current slot
        slot_start = (current_slot - 1) % 24
        slot_end = current_slot % 24
        print(f"\nCurrent Slot {current_slot:02d} (Hour {slot_start:02d}:00 to Hour {slot_end:02d}:00)")
        slot_df = df[df['Slot'] == current_slot]
        if not slot_df.empty:
            print(slot_df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))
        else:
            print("No entries")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Current Slot 12 (Hour 11:00 to Hour 12:00)
 Time Line ID Stop Point ID Direction
11:05     179    490005183E  Outbound
11:09     179    490005183E  Outbound
11:10     179    490005183E  Outbound
11:10     179    490005183E  Outbound
11:17     179    490005183E  Outbound
11:22     179    490005183E  Outbound
11:22     179    490005183E  Outbound
11:29     179    490005183E  Outbound
11:29     179    490005183E  Outbound
11:34     179    490005183E  Outbound
11:34     179    490005183E  Outbound
11:41     179    490005183E  Outbound
11:46     179    490005183E  Outbound
11:46     179    490005183E  Outbound
11:49     179    490005183E  Outbound
11:53     179    490005183E  Outbound
11:58     179    490005183E  Outbound
11:58     179    490005183E  Outbound


Fetching time table- User inputs--> line ID, stop point, direction, day of the week

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction, day_index):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable schedules
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])

        # Determine the schedule based on user input
        if day_index == 1:
            day_schedule = next((schedule for schedule in schedules if any(day in schedule.get('name', '').lower() for day in ['monday', 'tuesday', 'wednesday', 'thursday'])), None)
        elif day_index == 2:
            day_schedule = next((schedule for schedule in schedules if 'friday' in schedule.get('name', '').lower()), None)
        elif day_index == 3:
            day_schedule = next((schedule for schedule in schedules if 'saturday' in schedule.get('name', '').lower()), None)
        elif day_index == 4:
            day_schedule = next((schedule for schedule in schedules if 'sunday' in schedule.get('name', '').lower()), None)
        else:
            print("Invalid day index. Please enter a number between 1 and 4.")
            return

        if not day_schedule:
            print(f"No timetable available for selected day.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for entry in day_schedule.get('knownJourneys', []):
            timetable_entries.append({
                'Hour': int(entry['hour']),
                'Minute': int(entry['minute']),
                'Line ID': line_id,
                'Stop Point ID': stop_point_id,
                'Direction': direction.capitalize()
            })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Get current hour in BST
        bst_tz = pytz.timezone('Europe/London')
        current_time = datetime.now(bst_tz)
        current_hour = current_time.hour

        # Determine current slot
        current_slot = current_hour + 1
        if current_slot > 24:
            current_slot = 1

        # Display timetable for the selected day
        print(f"\nSelected Day: {day_schedule['name']}")
        print(f"Current Slot {current_slot:02d} (Hour {current_hour:02d}:00 to Hour {current_hour+1:02d}:00):")
        print("Timetable:")
        print(df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")

    # Ask user to select day of the week
    print("\nSelect Day of the Week:")
    print("1. Monday to Thursday")
    print("2. Friday")
    print("3. Saturday")
    print("4. Sunday")
    day_index = int(input("Enter the number corresponding to the day of the week: "))

    fetch_and_display_timetable(line_id, stop_point_id, direction, day_index)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Select Day of the Week:
1. Monday to Thursday
2. Friday
3. Saturday
4. Sunday
Enter the number corresponding to the day of the week: 1
No timetable available for selected day.


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction, day_of_week):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable schedules
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])

        # Determine the schedule based on user input
        day_schedule = None

        # Check if day of week is explicitly mentioned in API response
        for schedule in schedules:
            schedule_name = schedule.get('name', '').lower()
            if day_of_week.lower() in schedule_name:
                day_schedule = schedule
                break

        if not day_schedule:
            # Check for slot-based schedules like "Monday to Thursday" or "Mon-Fri"
            if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
                day_schedule = next((schedule for schedule in schedules if 'monday to thursday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'friday':
                day_schedule = next((schedule for schedule in schedules if 'friday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'saturday':
                day_schedule = next((schedule for schedule in schedules if 'saturday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'sunday':
                day_schedule = next((schedule for schedule in schedules if 'sunday' in schedule.get('name', '').lower()), None)
            else:
                print(f"No timetable available for {day_of_week}.")
                return

        if not day_schedule:
            print(f"No timetable available for {day_of_week}.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for entry in day_schedule.get('knownJourneys', []):
            timetable_entries.append({
                'Hour': int(entry['hour']),
                'Minute': int(entry['minute']),
                'Line ID': line_id,
                'Stop Point ID': stop_point_id,
                'Direction': direction.capitalize()
            })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Get current hour in BST
        bst_tz = pytz.timezone('Europe/London')
        current_time = datetime.now(bst_tz)
        current_hour = current_time.hour

        # Determine current slot
        current_slot = current_hour + 1
        if current_slot > 24:
            current_slot = 1

        # Display timetable for the selected day
        print(f"\nSelected Day: {day_schedule['name']}")
        print(f"Current Slot {current_slot:02d} (Hour {current_hour:02d}:00 to Hour {current_hour+1:02d}:00):")
        print("Timetable:")
        print(df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    day_of_week = input("Enter day of the week (e.g., Monday, Tuesday, ...): ")

    fetch_and_display_timetable(line_id, stop_point_id, direction, day_of_week)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound
Enter day of the week (e.g., Monday, Tuesday, ...): Wednesday
No timetable available for Wednesday.


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction, day_of_week):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable schedules
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])

        # Determine the schedule based on user input
        day_schedule = None

        # Check if day of week is explicitly mentioned in API response
        for schedule in schedules:
            schedule_name = schedule.get('name', '').lower()
            if day_of_week.lower() in schedule_name:
                day_schedule = schedule
                break

        if not day_schedule:
            # Check for slot-based schedules like "Monday to Thursday" or "Mon-Fri"
            if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
                day_schedule = next((schedule for schedule in schedules if 'monday to thursday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'friday':
                day_schedule = next((schedule for schedule in schedules if 'friday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'saturday':
                day_schedule = next((schedule for schedule in schedules if 'saturday' in schedule.get('name', '').lower()), None)
            elif day_of_week.lower() == 'sunday':
                day_schedule = next((schedule for schedule in schedules if 'sunday' in schedule.get('name', '').lower()), None)
            else:
                print(f"No timetable available for {day_of_week}.")
                return

        if not day_schedule:
            print(f"No timetable available for {day_of_week}.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for entry in day_schedule.get('knownJourneys', []):
            timetable_entries.append({
                'Hour': int(entry['hour']),
                'Minute': int(entry['minute']),
                'Line ID': line_id,
                'Stop Point ID': stop_point_id,
                'Direction': direction.capitalize()
            })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Get current hour in BST
        bst_tz = pytz.timezone('Europe/London')
        current_time = datetime.now(bst_tz)
        current_hour = current_time.hour

        # Determine current slot
        current_slot = current_hour + 1
        if current_slot > 24:
            current_slot = 1

        # Display timetable for the selected day
        print(f"\nSelected Day: {day_schedule['name']}")
        print(f"Current Slot {current_slot:02d} (Hour {current_hour:02d}:00 to Hour {current_hour+1:02d}:00):")
        print("Timetable:")
        print(df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    day_of_week = input("Enter day of the week (e.g., Monday, Tuesday, ...): ")

    fetch_and_display_timetable(line_id, stop_point_id, direction, day_of_week)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound
Enter day of the week (e.g., Monday, Tuesday, ...): Wednesday
No timetable available for Wednesday.


basic TT coding


In [ ]:
pip install requests tabulate


In [ ]:
import requests
import json

# Define the API endpoint URL
url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

# Send GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse JSON response
    timetable_data = response.json()

    # Extract timetable information
    schedules = timetable_data.get('timetable', {}).get('schedules', [])

    # Display timetable data
    for schedule in schedules:
        schedule_name = schedule.get('name', '')
        known_journeys = schedule.get('knownJourneys', [])

        print(f"Schedule: {schedule_name}")
        for journey in known_journeys:
            hour = journey.get('hour', '')
            minute = journey.get('minute', '')
            print(f"- Departure time: {hour}:{minute}")
        print()

else:
    print(f"Failed to fetch data: {response.status_code}")


In [ ]:
import requests
import json

# Define the API endpoint URL
url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

try:
    # Send GET request to the API endpoint
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        timetable_data = response.json()

        # Extract timetable information
        schedules = timetable_data.get('timetable', {}).get('schedules', [])

        if schedules:
            # Display timetable data
            for schedule in schedules:
                schedule_name = schedule.get('name', '')
                known_journeys = schedule.get('knownJourneys', [])

                print(f"Schedule: {schedule_name}")
                for journey in known_journeys:
                    hour = journey.get('hour', '')
                    minute = journey.get('minute', '')
                    print(f"- Departure time: {hour}:{minute}")
                print()
        else:
            print("No schedules found in the API response.")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")


No schedules found in the API response.


In [ ]:
import requests

# Define the API endpoint URL
url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

try:
    # Send GET request to the API endpoint
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        timetable_data = response.json()

        # Extract knownJourneys names
        known_journeys = timetable_data.get('timetable', {}).get('knownJourneys', [])

        # Print names of knownJourneys
        for journey in known_journeys:
            print(journey.get('name'))

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")


In [ ]:
import requests

# Define the API endpoint URL
url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

try:
    # Send GET request to the API endpoint
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        timetable_data = response.json()

        # Extract knownJourneys names
        known_journeys = timetable_data.get('timetable', {}).get('knownJourneys', [])

        if known_journeys:
            # Print names of knownJourneys
            for journey in known_journeys:
                print(journey.get('name'))
        else:
            print("No knownJourneys found in the API response.")

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")



No knownJourneys found in the API response.


In [ ]:
import requests

# Replace this URL with your actual API endpoint
url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

# Make a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse JSON response
    data = response.json()

    # Extract schedule names from the 'schedules' array
    schedules = data.get('schedules', [])

    # Iterate over each schedule and print its name
    for index, schedule in enumerate(schedules, start=1):
        name = schedule.get('name', '')
        print(f"{index}. {name}")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


In [ ]:
import json

# Sample JSON data
json_data = '''
{
    "$type": "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities",
    "name": "Sunday",
    "knownJourneys": [
        {
            "$type": "Tfl.Api.Presentation.Entities.KnownJourney, Tfl.Api.Presentation.Entities",
            "hour": "6",
            "minute": "42",
            "intervalId": 0
        },
        {
            "$type": "Tfl.Api.Presentation.Entities.KnownJourney, Tfl.Api.Presentation.Entities",
            "hour": "7",
            "minute": "30",
            "intervalId": 0
        }
    ]
}
'''

try:
    # Parse JSON data
    data = json.loads(json_data)

    # Access the 'name' field
    schedule_name = data['name']

    # Print the schedule name
    print(f"Schedule name: {schedule_name}")

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
except KeyError as e:
    print(f"KeyError: {e} - 'name' field not found in JSON")


Schedule name: Sunday


In [ ]:
import requests

url = "https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound"

try:
    # Make a GET request to the API
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        data = response.json()

        # Print the entire JSON response for inspection
        print(data)

        # Extract schedule information from the 'schedules' array
        schedules = data.get('schedules', [])

        # Print schedule information
        if schedules:
            print("Schedule Information:")
            for schedule in schedules:
                name = schedule.get('name', '')
                print(f"Schedule Name: {name}")

                # Extract and print known journeys
                known_journeys = schedule.get('knownJourneys', [])
                if known_journeys:
                    print("Known Journeys:")
                    for journey in known_journeys:
                        hour = journey.get('hour', '')
                        minute = journey.get('minute', '')
                        print(f"- Time: {hour}:{minute}")
                else:
                    print("No known journeys found for this schedule.")

                print()  # Add a newline for better readability

        else:
            print("No schedules found.")

    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except ValueError as e:
    print(f"Error parsing JSON: {e}")


{'$type': 'Tfl.Api.Presentation.Entities.TimetableResponse, Tfl.Api.Presentation.Entities', 'lineId': '179', 'lineName': '179', 'direction': 'outbound', 'stations': [{'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'stationId': '490G000007', 'icsId': '1020390', 'topMostParentId': '490G000007', 'modes': ['bus'], 'stopType': 'NaptanOnstreetBusCoachStopPair', 'lines': [{'$type': 'Tfl.Api.Presentation.Entities.Identifier, Tfl.Api.Presentation.Entities', 'id': '179', 'name': '179', 'uri': '/Line/179', 'type': 'Line', 'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, Tfl.Api.Presentation.Entities'}, 'routeType': 'Unknown', 'status': 'Unknown'}, {'$type': 'Tfl.Api.Presentation.Entities.Identifier, Tfl.Api.Presentation.Entities', 'id': '397', 'name': '397', 'uri': '/Line/397', 'type': 'Line', 'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, Tfl.Api.Presentation.Entities'}, 'routeType': 'Unknown', 'status': 'Unknown'}], 'status': T

In [ ]:
import json

# Example API response (JSON data as string)
api_response = '''
{
  "schedules": [
    {
      "$type": "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities",
      "name": "Mon-Fri Non-Schooldays",
      "knownJourneys": [
        {
          "$type": "Tfl.Api.Presentation.Entities.KnownJourney, Tfl.Api.Presentation.Entities",
          "hour": "4",
          "minute": "57",
          "intervalId": 0
        },
        {
          "$type": "Tfl.Api.Presentation.Entities.KnownJourney, Tfl.Api.Presentation.Entities",
          "hour": "5",
          "minute": "17",
          "intervalId": 0
        }
      ]
    },
    {
      "$type": "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities",
      "name": "Mon-Fri Schooldays",
      "knownJourneys": []
    },
    {
      "$type": "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities",
      "name": "Saturday",
      "knownJourneys": []
    },
    {
      "$type": "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities",
      "name": "Sunday",
      "knownJourneys": []
    }
  ]
}
'''

# Parse the JSON response
data = json.loads(api_response)

# Extract names of schedules
schedule_names = [schedule['name'] for schedule in data['schedules']]

# Print the schedule names
for name in schedule_names:
    print(name)


Mon-Fri Non-Schooldays
Mon-Fri Schooldays
Saturday
Sunday


In [ ]:
import requests

# API URL
url = 'https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound'

try:
    # Send GET request to the API
    response = requests.get(url)

    # Raise an exception for HTTP errors
    response.raise_for_status()

    # Parse JSON response
    data = response.json()

    # Print the raw JSON response (for debugging)
    print(response.text)

    # Initialize an empty list to store schedule names
    schedule_names = []

    # Traverse through the JSON structure to find schedule names
    if 'timetable' in data and 'routes' in data['timetable']:
        for route in data['timetable']['routes']:
            if 'stationIntervals' in route:
                for interval in route['stationIntervals']:
                    if 'stopLetter' in interval:
                        for stop_letter in interval['stopLetter']:
                            if 'knownJourneys' in stop_letter:
                                for journey in stop_letter['knownJourneys']:
                                    if "$type" in journey and journey["$type"] == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities":
                                        schedule_names.append(journey["name"])

    # Print the schedule names
    if schedule_names:
        for name in schedule_names:
            print(name)
    else:
        print("No schedule names found.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except Exception as e:
    print(f"Error: {e}")


{"$type":"Tfl.Api.Presentation.Entities.TimetableResponse, Tfl.Api.Presentation.Entities","lineId":"179","lineName":"179","direction":"outbound","stations":[{"$type":"Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities","stationId":"490G000007","icsId":"1020390","topMostParentId":"490G000007","modes":["bus"],"stopType":"NaptanOnstreetBusCoachStopPair","lines":[{"$type":"Tfl.Api.Presentation.Entities.Identifier, Tfl.Api.Presentation.Entities","id":"179","name":"179","uri":"/Line/179","type":"Line","crowding":{"$type":"Tfl.Api.Presentation.Entities.Crowding, Tfl.Api.Presentation.Entities"},"routeType":"Unknown","status":"Unknown"},{"$type":"Tfl.Api.Presentation.Entities.Identifier, Tfl.Api.Presentation.Entities","id":"397","name":"397","uri":"/Line/397","type":"Line","crowding":{"$type":"Tfl.Api.Presentation.Entities.Crowding, Tfl.Api.Presentation.Entities"},"routeType":"Unknown","status":"Unknown"}],"status":true,"id":"490G000007","name":"Newlands Road","lat":51.624

In [ ]:
import requests

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names=[]):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names.append(data['name'])
        for key, value in data.items():
            extract_schedule_names(value, schedule_names)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names)
    return schedule_names

# URL of the API
url = 'https://api.tfl.gov.uk/Line/179/Timetable/490005183E?direction=outbound'

# Fetching data
data = fetch_data(url)

# Extracting schedule names
schedule_names = extract_schedule_names(data)

# Printing the extracted schedule names
print(schedule_names)


['Mon-Fri Non-Schooldays', 'Mon-Fri Schooldays', 'Saturday', 'Sunday']


In [ ]:
Code printing the schedule names from the time table API

In [ ]:
import requests

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names=[]):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names.append(data['name'])
        for key, value in data.items():
            extract_schedule_names(value, schedule_names)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names)
    return schedule_names

# Main function to get user input and print the schedule names
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Construct the URL
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'

    # Fetching data
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names = extract_schedule_names(data)

    # Printing the extracted schedule names as a numbered list
    print("Schedule Names:")
    for i, name in enumerate(schedule_names, start=1):
        print(f"{i}. {name}")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 22
Enter the stop point ID: 490012375E
Enter the direction (outbound/inbound): outbound
Schedule Names:
1. Friday
2. Monday to Thursday
3. Saturday
4. Sunday


In [ ]:
import requests

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Construct the URL
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'

    # Fetching data
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Printing the extracted schedule names as a numbered list
    print("Schedule Names:")
    schedule_names = list(schedule_names_dict.keys())
    for i, name in enumerate(schedule_names, start=1):
        print(f"{i}. {name}")

    # Asking the user to select a schedule
    selected_number = int(input("Enter the number corresponding to the schedule name: "))
    selected_schedule_name = schedule_names[selected_number - 1]

    # Printing the timetable for the selected schedule
    print(f"\nTimetable for {selected_schedule_name}:")
    timetable = schedule_names_dict[selected_schedule_name]
    for journey in timetable:
        print(f"Hour: {journey['hour']}, Minute: {journey['minute']}")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 141
Enter the stop point ID: 490015195M
Enter the direction (outbound/inbound): outbound
Schedule Names:
1. Friday
2. Monday to Thursday
3. Saturday
4. Sunday
Enter the number corresponding to the schedule name: 2

Timetable for Monday to Thursday:
Hour: 4, Minute: 50
Hour: 5, Minute: 2
Hour: 5, Minute: 14
Hour: 5, Minute: 26
Hour: 5, Minute: 38
Hour: 5, Minute: 50
Hour: 5, Minute: 59
Hour: 6, Minute: 8
Hour: 6, Minute: 14
Hour: 6, Minute: 21
Hour: 6, Minute: 27
Hour: 6, Minute: 33
Hour: 6, Minute: 39
Hour: 6, Minute: 45
Hour: 6, Minute: 51
Hour: 6, Minute: 58
Hour: 7, Minute: 4
Hour: 7, Minute: 10
Hour: 7, Minute: 16
Hour: 7, Minute: 22
Hour: 7, Minute: 28
Hour: 7, Minute: 34
Hour: 7, Minute: 40
Hour: 7, Minute: 46
Hour: 7, Minute: 53
Hour: 7, Minute: 59
Hour: 8, Minute: 6
Hour: 8, Minute: 12
Hour: 8, Minute: 17
Hour: 8, Minute: 22
Hour: 8, Minute: 27
Hour: 8, Minute: 32
Hour: 8, Minute: 37
Hour: 8, Minute: 42
Hour: 8, Minute: 47
Hour: 8, Minute: 52
Hour: 8, Minute: